<a href="https://colab.research.google.com/github/laribar/TechChallenge2/blob/main/TechChallenge_2_Rev01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [116]:
#!pip install yfinance --upgrade --no-cache-dir


In [117]:
# Análise de dados e manipulação
import pandas as pd
import numpy as np

# Visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns


from statsmodels.tsa.stattools import adfuller

# Machine Learning
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
# pegar os dados
import yfinance as yf
# Manipulação de datas
from datetime import datetime
from pandas.tseries.offsets import BDay

# Verificação de versões (opcional)
import sklearn
print(f"Scikit-learn version: {sklearn.__version__}")

Scikit-learn version: 1.6.1


In [118]:
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


In [119]:
import warnings
import logging
from tqdm import tqdm
import yfinance as yf
import pandas as pd
from datetime import datetime

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Ignorar warnings desnecessários
warnings.filterwarnings('ignore')

print("Baixando dados...")

# Baixar os dados do Ibovespa
df = yf.download('^BVSP', start='1994-07-01', end=datetime.now().strftime('%Y-%m-%d'))

# Verificar se os dados foram baixados corretamente
if df.empty:
    logging.warning("Falha ao baixar os dados do Ibovespa. Verifique a conexão ou a disponibilidade do Yahoo Finance.")
else:
    logging.info(f"Dados baixados com sucesso! Total de registros: {df.shape[0]}")

# Preencher valores ausentes e garantir que o índice seja datetime
df = df.ffill()
df.index = pd.to_datetime(df.index)


[*********************100%***********************]  1 of 1 completed

Baixando dados...


In [120]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7591 entries, 1994-07-04 to 2025-02-26
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   (Close, ^BVSP)   7591 non-null   float64
 1   (High, ^BVSP)    7591 non-null   float64
 2   (Low, ^BVSP)     7591 non-null   float64
 3   (Open, ^BVSP)    7591 non-null   float64
 4   (Volume, ^BVSP)  7591 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 355.8 KB


In [121]:
# Criar Features
for i in range(1, 6):
    df[f'Lag{i}'] = df['Close'].shift(i)

df['Daily_Return'] = df['Close'].pct_change()
df['MA_7'] = df['Close'].rolling(window=7, min_periods=1).mean()
df['MA_30'] = df['Close'].rolling(window=30, min_periods=1).mean()
df['Volatility'] = df['Close'].pct_change().rolling(window=7, min_periods=1).std()
df['Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))

# Bandas de Bollinger
df['BB_Mid'] = df['Close'].rolling(window=20, min_periods=1).mean()
std_dev = df['Close'].rolling(window=20, min_periods=1).std().squeeze()
df['BB_Upper'] = df['BB_Mid'] + (std_dev * 2)
df['BB_Lower'] = df['BB_Mid'] - (std_dev * 2)

# RSI (Relative Strength Index)
delta = df['Close'].diff(1)
gain = (delta.where(delta > 0, 0)).rolling(window=14, min_periods=1).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14, min_periods=1).mean()
rs = gain / loss
df['RSI'] = 100 - (100 / (1 + rs))

# Armazena a variável alvo antes do dropna()
y = df['Close'].copy()

# Remover NaNs
df.dropna(inplace=True)

# Criar X e y mantendo os índices alinhados
X = df.drop(columns=['Close'])
y = y.loc[X.index]  # Mantém y alinhado com X

# Separação treino e teste baseada no tempo
train_size = int(len(df) * 0.8)
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]


In [122]:
print(f"Linhas antes do dropna: {len(df)}")
df.dropna(inplace=True)
print(f"Linhas após o dropna: {len(df)}")


Linhas antes do dropna: 7586
Linhas após o dropna: 7586


In [123]:
#parte 3


def test_stationarity(series, threshold=0.05):
    result = adfuller(series.dropna())  # Remover NaNs antes do teste
    print(f'ADF Statistic: {result[0]:.4f}')
    print(f'p-value: {result[1]:.4f}')
    return result[1] <= threshold

def make_stationary(y_train, y_test, X_train, X_test):
    if not test_stationarity(y_train):
        y_train = y_train.diff().dropna()
        y_test = y_test.diff().dropna()
        print("Série não era estacionária. Aplicada diferenciação de primeira ordem.")

        # Ajustando X para manter alinhamento com y
        X_train = X_train.loc[y_train.index]
        X_test = X_test.loc[y_test.index]

    return y_train, y_test, X_train, X_test

# Aplicar ajuste de estacionariedade
y_train, y_test, X_train, X_test = make_stationary(y_train, y_test, X_train, X_test)


ADF Statistic: -0.1832
p-value: 0.9405
Série não era estacionária. Aplicada diferenciação de primeira ordem.


In [124]:
# Parte 4 - Normalização para Modelos Lineares (evitando data leakage)

scaler = StandardScaler()

# Fit apenas no conjunto de treino e transformação nos dois conjuntos
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [127]:
# Ajustando Modelos

# Random Forest NÃO precisa de escalonamento, então usamos os dados brutos
best_params_rf = {'n_estimators': 210, 'max_depth': 9, 'random_state': 42}  # Garantir reprodutibilidade
rf_model = RandomForestRegressor(**best_params_rf)

# Treinamento do modelo
rf_model.fit(X_train, y_train)  # Treinamos com os dados originais, sem escalonamento

# Previsões
y_pred_rf = rf_model.predict(X_test)  # Também sem escalonamento
y_pred_rf = pd.Series(y_pred_rf, index=y_test.index)  # Garantir que o índice seja o mesmo de y_test

# Se aplicamos diferenciação, precisamos reverter os valores previstos
if "diff" in y_train.index.name.lower() if y_train.index.name else False:
    y_pred_rf = y_pred_rf.cumsum() + y.iloc[train_size - 1]  # Usar o último valor antes da diferença

# Avaliação do Modelo
mse = mean_squared_error(y_test, y_pred_rf)
mae = mean_absolute_error(y_test, y_pred_rf)
r2 = r2_score(y_test, y_pred_rf)

# Exibir métricas com formatação
print(f'MSE: {mse:.2f}')
print(f'MAE: {mae:.2f}')
print(f'R²: {r2:.4f}')


MSE: 261140.55
MAE: 277.65
R²: 0.8892
